In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
%matplotlib inline

In [12]:
sub = 'sub-04'
ses = 'ses-01'
battery = 'camcan'
task = 'emotional-memory'

main_dir = '/home/sshankar/ibc/analysis_pipeline/ibc_main/neurospin_data/info/'
datadir = os.path.join(main_dir, sub, battery, ses)
os.chdir(datadir)
csv_files = sorted(glob.glob(task + '*.txt'))
tsv_files = []
i = 0
# for i in range(len(csv_files)):
tsv_files.append(task + '_' + sub + '_run-0' + str(i+1) + '.tsv')

In [14]:
csv_files

['emotional-memory_sub-07_run-01_1614692251.txt',
 'emotional-memory_sub-07_run-02_1614692968.txt']

In [13]:
# In the log file run start is the file line.
# To extract it read in the first line by itself and then 
# read the rest of the file as a DataFrame.
line1 = pd.read_csv(os.path.join(datadir, csv_files[i]), sep=':', header=None, nrows=1)
run_start = line1[1][0]

In [14]:
run_start

595.159

In [15]:
file_df = pd.read_csv(os.path.join(datadir, csv_files[i]), header=1, sep=',', skipinitialspace=True)
# file_df = pd.read_csv(os.path.join(datadir, csv_files[0]), sep=',')

In [7]:
file_df

,img_valence,img_file,img_onset,img_actual_onset,img_onset_end,obj_file,obj_onset,obj_actual_onset,obj_onset_end,resp_key,resp_onset
0,3,\Negative\9040.JPG,1166.1031,1166.1031,1166.1031,\Objects\firetruck.bmp,1168.1069,1168.1069,1168.1069,NaN,-9.0000
1,2,\Neutral\7100.JPG,1176.5751,1176.5751,1176.5752,\Objects\curtain.bmp,1178.5800,1178.5800,1178.5800,y,1183.5800
2,2,\Neutral\7186.JPG,1186.7121,1186.7121,1186.7122,\Objects\hairbrush.bmp,1188.7173,1188.7173,1188.7174,y,1194.3124
3,1,\Positive\2311.JPG,1196.8420,1196.8420,1196.8420,\Objects\chimney.bmp,1198.8468,1198.8468,1198.8469,y,1204.1515
4,3,\Negative\9433.JPG,1206.9752,1206.9752,1206.9752,\Objects\marbles.bmp,1208.9802,1208.9802,1208.9802,y,1213.3836
5,1,\Positive\8496.JPG,1217.1100,1217.1100,1217.1100,\Objects\lantern.bmp,1219.1149,1219.1149,1219.1149,y,1224.1923
6,3,\Negative\9265.JPG,1227.2642,1227.2642,1227.2642,\Objects\hand.bmp,1229.2689,1229.2689,1229.2689,NaN,-9.0000
7,2,\Neutral\2749.JPG,1237.3970,1237.3970,1237.3971,\Objects\bowtie.bmp,1239.3990,1239.3990,1239.3990,y,1242.7204
8,1,\Positive\7200.JPG,1247.5250,1247.5250,1247.5250,\Objects\goggles.bmp,1249.5302,1249.5302,1249.5302,y,1253.2284
9,1,\Positive\4603.JPG,1257.6567,1257.6567,1257.6567,\Objects\drill.bmp,1259.6627,1259.6627,1259.6628,y,1263.7164


In [16]:
# Find the trial-related rows
# trials = file_df['img_valence'] != 'Scanner'
valence = file_df['img_valence']
negative_trials = file_df['img_valence'].isin(['3'])
neutral_trials = file_df['img_valence'].isin(['2'])
positive_trials = file_df['img_valence'].isin(['1'])

In [43]:
print(len(trials), sum(trials))

60 60


In [44]:
positive_trials

0      True
1      True
2     False
3     False
4     False
5     False
6      True
7     False
8     False
9     False
10     True
11    False
12    False
13     True
14    False
15     True
16    False
17     True
18    False
19    False
20    False
21     True
22    False
23    False
24    False
25     True
26    False
27    False
28    False
29    False
30     True
31    False
32    False
33    False
34     True
35    False
36     True
37    False
38     True
39     True
40    False
41     True
42     True
43    False
44    False
45    False
46     True
47    False
48    False
49     True
50    False
51     True
52     True
53     True
54     True
55    False
56     True
57    False
58    False
59    False
Name: img_valence, dtype: bool

In [45]:
len(file_df)

60

In [17]:
# Trial/Image onsets are in 'img_onset'
img_onsets = file_df['img_onset'].astype(float)

# Object onsets are in 'obj_onset'
obj_onsets = file_df['obj_onset'].astype(float)

# Concatenate the two to get the onsets array
trial_onsets = np.vstack((img_onsets, obj_onsets)).flatten('F')

## Create durations array
# In the task, the image is presented for 8 s. 2 s after image onset, 
# the object is presented, and stays on screen for the remaining 6 s.
img_durations = np.repeat(8,len(file_df))
obj_durations = np.repeat(6,len(file_df))
# Interlace the two durations to create the durations array
durations = (np.vstack((img_durations,obj_durations))).flatten('F')

In [14]:
img_onsets

0     104773.4525
1     104783.9029
2     104794.0330
3     104804.1706
4     104814.3089
5     104824.4452
6     104834.5843
7     104844.7247
8     104854.8634
9     104865.0041
10    104875.1514
11    104885.2894
12    104895.4306
13    104905.5683
14    104915.7023
15    104925.8328
16    104935.9714
17    104946.1110
18    104956.2485
19    104966.3876
20    104976.5247
21    104986.6578
22    104996.7972
23    105006.9333
24    105017.0735
25    105027.2131
26    105037.3531
27    105047.4957
28    105057.6356
29    105067.7756
30    105077.9148
31    105088.0996
32    105098.2367
33    105108.3768
34    105118.5091
35    105128.6471
36    105138.7888
37    105148.9282
38    105159.0723
39    105169.2083
40    105179.3496
41    105189.4868
42    105199.6253
43    105209.7634
44    105219.9033
45    105230.0381
46    105240.1744
47    105250.3139
48    105260.4539
49    105270.5918
50    105280.7327
51    105290.8719
52    105301.0124
53    105311.1537
54    105321.2919
55    1053

In [18]:
## Finally, create the trial_types array. This will be an interlacing
## of the 'img_valence' vector for images and 'object' for objects
img_types = np.array(np.ones(len(file_df)), dtype=np.object_)
img_types[negative_trials] = 'negative_image'
img_types[neutral_trials] = 'neutral_image'
img_types[positive_trials] = 'positive_image'

In [19]:
obj_types = np.repeat('object', len(file_df))

In [20]:
trial_types = np.vstack((img_types,obj_types)).flatten('F')

In [21]:
# Create a DataFrame to save to events file
df = pd.DataFrame({'onset':(trial_onsets-run_start), 'duration':durations, 'trial_type':trial_types})
df.to_csv(os.path.join(datadir, tsv_files[0]), sep='\t', float_format='%0.3f', index=False)